In [1]:
# Json Parser loaded

In [3]:
from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser()
parser.get_format_instructions()

'Return a JSON object.'

In [4]:
# Retry Parser

In [21]:
from langchain.output_parsers import RetryWithErrorOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
parser = RetryWithErrorOutputParser.from_llm(parser=JsonOutputParser(), llm=ChatOpenAI())

In [22]:
question = "가장 큰 대륙은?"
ai_response = "아시아입니다."

try:
    result = parser.parse_with_prompt(ai_response, question)
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: 'str' object has no attribute 'to_string'


In [23]:
# pydantic

In [24]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, model_validator

In [25]:
model = ChatOpenAI(model='gpt-4o', temperature=0.0)

In [30]:
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict)->dict:
        setup = values.get('setup', '')
        if not setup.endswith('?'):
            raise ValueError('잘못된 질문 형식입니다! 질문은 "?"로 끝나야 합니다.')
        return values
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해 주세요.\n{format_instructions}\n질문:{query}\n",
    input_variables=['query'],
    partial_variables={'format_instructions':parser.get_format_instructions()}
)
chain = prompt|model|StrOutputParser()|parser

try:
    result = chain.invoke({'query':'부동산에 관련하여 금융 조언을 받을 수 있게 질문하라'})
    print(result)
except Exception as e:
    print(f'오류 발생 : {e}')

setup='부동산 투자에 관심이 있습니다. 현재 시장 상황을 고려할 때, 부동산에 투자하는 것이 좋은 선택일까요?' advice='부동산 투자는 장기적인 관점에서 안정적인 수익을 제공할 수 있지만, 시장 상황에 따라 변동성이 있을 수 있습니다. 현재 금리, 지역 경제 성장 전망, 부동산 가격 추세 등을 고려하여 투자 결정을 내리는 것이 중요합니다. 전문가와 상담하여 포트폴리오를 다각화하고 리스크를 관리하는 전략을 세우는 것이 좋습니다.'


In [35]:
class CodingAdvice(BaseModel):
    question: str = Field(description="코딩 초보자 상황에서 오류를 해결하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 기술적인 답변")
    @model_validator(mode='before')
    @classmethod
    def question_ends_with_question_mark(cls, values: dict)->dict:
        question = values.get('question', '')
        if not question.endswith('?'):
            raise ValueError('잘못된 질물 형식입니다! 질문은 "?"로 끝나야합니다.')
        return values
parser = PydanticOutputParser(pydantic_object=CodingAdvice)
prompt = PromptTemplate(
    template="다음 코딩 관련 질문에 답변해주세요.\n{format_instruction}\n질문:{query}\n",
    input_variables=['query'],
    partial_variables={'format_instruction':parser.get_format_instructions}
)
chain = prompt|ChatOpenAI(model='gpt-4o-mini')|StrOutputParser()|parser
try:
    result = chain.invoke({'query':'코딩에 관련하여 오류를 해결받기 위한 조언을 받을 수 있게 질문하라'})
    print(result)
except Exception as e:
    print(f'오류 발생 : {e}')

question='코딩에서 발생하는 오류를 어떻게 디버깅하고 해결할 수 있을까요?' advice='오류 메시지를 잘 읽고, 문제가 발생하는 코드 주위를 주의 깊게 살펴보세요. 주석을 추가하거나 출력문을 사용하여 프로그램의 흐름을 추적하고, 각 변수의 값이 예상대로인지 확인하세요. 또한, Google이나 Stack Overflow에서 유사한 문제를 검색해 보는 것도 좋은 방법입니다.'


In [42]:
# SimpleJsonOutputParser

In [36]:
from langchain.output_parsers.json import SimpleJsonOutputParser
json_prompt = PromptTemplate.from_template('다음 질문에 대한 답변이 포함된 JSON 객체를 반환하십시오: {question}')
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt|model|json_parser

In [41]:
for i in json_chain.stream({'question':'비트코인에 대한 잛은 한 문장 설명.'}):
    print(i)

{}
{'description': ''}
{'description': '비'}
{'description': '비트'}
{'description': '비트코'}
{'description': '비트코인은'}
{'description': '비트코인은 분'}
{'description': '비트코인은 분산'}
{'description': '비트코인은 분산된'}
{'description': '비트코인은 분산된 디'}
{'description': '비트코인은 분산된 디지털'}
{'description': '비트코인은 분산된 디지털 화'}
{'description': '비트코인은 분산된 디지털 화폐'}
{'description': '비트코인은 분산된 디지털 화폐로'}
{'description': '비트코인은 분산된 디지털 화폐로,'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 개인'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 개인 간'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 개인 간의'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 개인 간의 거래'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 개인 간의 거래를'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 개인 간의 거래를 가능'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 개인 간의 거래를 가능하게'}
{'description': '비트코인은 분산된 디지털 화폐로, 중앙 기관 없이 개인 간의 거래를 가능하게 합니다'}
{'de

In [43]:
# JsonOutputParser

In [44]:
from langchain_core.output_parsers import JsonOutputParser
model = ChatOpenAI(temperature=0)

In [45]:
class FinancialAdvice(BaseModel):
    setup: str = Field(description='금융 조언 상황을 설정하기 위한 질문')
    advice: str = Field(description='질문을 해결하기 위한 금융 답변')

In [46]:
parser = JsonOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template='다음 금융 관련 질문에 답변해 주세요.\n{format_instructions}\n{query}\n',
    input_variables=['query'],
    partial_variables={'format_instructions':parser.get_format_instructions()},
)

In [47]:
chain = prompt | model | parser
chain.invoke({'query':'부동산에 관련하여 금융 조언을 받을 수 있게 질문하라.'})

{'setup': '부동산 투자를 위해 어떤 금융 상품을 활용하는 것이 좋을까요?',
 'advice': '부동산 투자를 위해는 부동산 투자 전문가와 상담하고, 금융 상품으로는 부동산 투자 전용 대출을 고려해보세요. 또한 부동산 투자에 따른 세금 혜택과 리스크 관리에 대해도 충분히 고려해야 합니다.'}